# PyTorch Tutorial - Part 6
> "Part 6 of the PyTorch tutorial"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Scott Wolf
- categories: [PyTorch, jupyter]

# Putting it together with optimization!

The first cell is just code from the previous examples along 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [14]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Training loop

Now we have to setup a training loop! We need to explicitly zero out the gradients on each iteration. We then need to backpropagate the prediction loss using `loss.backward()` and then just step using `optimizer.step()`.

In [28]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(
        dataloader
    ):  # Batch access cause we wrapped the dset with dataloader
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [29]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (
                (pred.argmax(1) == y).type(torch.float).sum().item()
            )  # Get argmax along column since rows are the batch inputs

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 5.049222  [    0/60000]
loss: 4.444196  [ 6400/60000]
loss: 4.470216  [12800/60000]
loss: 3.558834  [19200/60000]
loss: 4.334649  [25600/60000]
loss: 4.048509  [32000/60000]
loss: 4.656027  [38400/60000]
loss: 4.429307  [44800/60000]
loss: 3.571862  [51200/60000]
loss: 4.724256  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 4.471746 

Epoch 2
-------------------------------
loss: 4.211938  [    0/60000]
loss: 4.383227  [ 6400/60000]
loss: 3.944262  [12800/60000]
loss: 3.761158  [19200/60000]
loss: 4.501288  [25600/60000]
loss: 4.550036  [32000/60000]
loss: 3.987212  [38400/60000]
loss: 5.760640  [44800/60000]
loss: 4.641942  [51200/60000]
loss: 5.218049  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 4.507261 

Epoch 3
-------------------------------
loss: 3.803559  [    0/60000]
loss: 4.146823  [ 6400/60000]
loss: 3.874949  [12800/60000]


In [ ]:
# Lets save the model in .pth format!
model = torch.save(model, "model.pth")